# Fitness function

In [1]:
import scipy.io as scio
import numpy as np

### Change the function to calculate fitness here

In each GNBG problem file, the author also provides the optimal value along with the corresponding solution. Thereby, we can check whether the fitness function code is correct.

In [2]:
def Transform(X, Alpha, Beta):
    Y = X.copy()
    tmp = (X > 0)
    Y[tmp] = np.log(X[tmp])
    Y[tmp] = np.exp(Y[tmp] + Alpha[0] * (np.sin(Beta[0] * Y[tmp]) + np.sin(Beta[1] * Y[tmp])))
    
    tmp = (X < 0)
    Y[tmp] = np.log(-X[tmp])
    Y[tmp] = -np.exp(Y[tmp] + Alpha[1] * (np.sin(Beta[2] * Y[tmp]) + np.sin(Beta[3] * Y[tmp])))
    
    return Y

### Get parameters

# Algorithm

### Here, two algorithms MFEA and DE are temporarily presented. The MFEA code here has been verified, while the DE code is rewritten from the official matlab code of the organizers.

## 1. MFEA
MFEA is a multifactorial optimization algorithm, so you should pass at least 2 functions to it to process.

Note that, in this MFEA code, each individual is a list , while the fitness_of_ind function receives an np.array as an argument, so you need to pass the np.array version of the individual when using this code.

In [3]:
class GNBG:
    def __init__(self, i):
        # get file path './fi.mat/' 
        self.idx = i
        self.file_path = './f' + str(i) + '.mat'
        self.GNBG = self.get_GNBG_from_path(self.file_path)
        self.LB = self.GNBG['MinCoordinate'][0][0][0][0]
        self.UB = self.GNBG['MaxCoordinate'][0][0][0][0]
        self.dim = self.GNBG['Dimension'][0][0][0][0]
        self.num_com = self.GNBG['o'][0][0][0][0]
        self.opt_value = self.GNBG['OptimumValue'][0][0][0][0]
        self.opt_position = self.GNBG['OptimumPosition'][0][0]
        self.FE = 0
        self.max_FE = 0
    def get_GNBG_from_path(self, file_path):
        data = scio.loadmat(file_path)
        GNBG = data['GNBG']
        print("ID:", self.idx)
        print("Optimal value: ", GNBG['OptimumValue'][0][0][0][0])
        # print("Num components: ", GNBG['o'][0][0][0][0])
        # print("Dimension: ", GNBG['Dimension'][0][0][0][0])
        # print("LB: ", GNBG['MinCoordinate'][0][0][0][0])
        # print("UB: ", GNBG['MaxCoordinate'][0][0][0][0])
        return GNBG
    def check(self):
        ind = self.fitness_of_ind(self.opt_position)
        val = self.opt_value
        print("Fitness of our code", ind)
        print("Fitness of the author", val)
    def fitness_of_pop(self, X):
        '''
        Input : A matrix of size (N,d), 
        N: number of individuals, d: number of dimensions, 
        kth row (1<=k<=N) represents kth individual vector.
        Output: a matrix (N,1) representing the fitness of N individuals.

        '''
        ### read the MATLAB file and call a few indices to get the same shape as the original competition file
        num_com = self.GNBG['o'][0][0][0][0]
        dim = self.GNBG['Dimension'][0][0][0][0]
        min_pos = self.GNBG['Component_MinimumPosition'][0][0].reshape(num_com, dim)
        rot_mat = self.GNBG['RotationMatrix'][0][0].reshape(dim,dim,num_com)
        sigma = self.GNBG['ComponentSigma'][0,0].reshape(num_com,1)
        hehe = self.GNBG['Component_H'][0,0].reshape(num_com,dim)
        mu = self.GNBG['Mu'][0][0].reshape(num_com,2)
        omega = self.GNBG['Omega'][0][0].reshape(num_com,4)
        lamb = self.GNBG['lambda'][0][0].reshape(num_com,1)
        SolutionNumber, _ = X.shape
        result = np.empty((SolutionNumber, 1))
        ### Converts the input vector format and returns the fitness list of individuals
        for jj in range(SolutionNumber):
            self.FE = self.FE + 1
            x = X[jj, :].reshape(-1,1)
            f = np.empty((1, num_com))
            
            for k in range(num_com):
                inp = x - min_pos[k,:].reshape(-1,1)
                a = Transform( inp.T  @  rot_mat[:,:,k] , mu[k,:], omega[k,:])
                b = Transform(rot_mat[:,:,k] @ inp, mu[k,:], omega[k,:] )
                f[0, k] = sigma[k] + (np.abs(a @ np.diag(hehe[k, :]) @ b))**lamb[k]

            result[jj] = np.min(f)
            
            # if GNBG['FE'] > GNBG['MaxEvals']:
            #     return result, GNBG
            
            # GNBG['FE'] += 1
            # GNBG['FEhistory'][GNBG['FE']] = result[jj]
            
            # if GNBG['BestFoundResult'] > result[jj]:
            #     GNBG['BestFoundResult'] = result[jj]
                
            # if (abs(GNBG['FEhistory'][GNBG['FE']] - GNBG['OptimumValue']) < GNBG['AcceptanceThreshold'] and
            #         np.isinf(GNBG['AcceptanceReachPoint'])):
            #     GNBG['AcceptanceReachPoint'] = GNBG['FE']
        
        return result
    def fitness_of_ind(self, X):
        num_com = self.GNBG['o'][0][0][0][0]
        dim = self.GNBG['Dimension'][0][0][0][0]
        min_pos = self.GNBG['Component_MinimumPosition'][0][0].reshape(num_com, dim)
        rot_mat = self.GNBG['RotationMatrix'][0][0].reshape(dim,dim,num_com)
        sigma = self.GNBG['ComponentSigma'][0,0].reshape(num_com,1)
        hehe = self.GNBG['Component_H'][0,0].reshape(num_com,dim)
        mu = self.GNBG['Mu'][0][0].reshape(num_com,2)
        omega = self.GNBG['Omega'][0][0].reshape(num_com,4)
        lamb = self.GNBG['lambda'][0][0].reshape(num_com,1)
        ### Converts the input vector format and returns the fitness list of individuals
        x = X.reshape(-1,1)
        f = np.empty((1, num_com))
        for k in range(num_com):
            inp = x - min_pos[k, :].reshape(-1,1)
            a = Transform( inp.T  @  rot_mat[:,:,k],mu[k, :],omega[k,:])
            b = Transform(rot_mat[:,:,k] @ inp, mu[k, :], omega[k, :] )
            f[0, k] = sigma[k] + (np.abs(a @ np.diag(hehe[k, :]) @ b))**lamb[k]


        result = np.min(f)
            
            # if GNBG['FE'] > GNBG['MaxEvals']:
            #     return result, GNBG
            
            # GNBG['FE'] += 1
            # GNBG['FEhistory'][GNBG['FE']] = result[jj]
            
            # if GNBG['BestFoundResult'] > result[jj]:
            #     GNBG['BestFoundResult'] = result[jj]
                
            # if (abs(GNBG['FEhistory'][GNBG['FE']] - GNBG['OptimumValue']) < GNBG['AcceptanceThreshold'] and
            #         np.isinf(GNBG['AcceptanceReachPoint'])):
            #     GNBG['AcceptanceReachPoint'] = GNBG['FE']
        
        return result
        
    def fitness_mfea(self, X):
        return self.fitness_of_ind(self.LB + np.array(X)*(self.UB-self.LB))

In [4]:
function_list = [GNBG(i).fitness_mfea for i in [1,2]]

ID: 1
Optimal value:  -1081.9837994003399
ID: 2
Optimal value:  -703.1328146165181


In [5]:
from MFEAcode import MFEA 

for i in [20]:
    function_list = [GNBG(i).fitness_mfea, GNBG(i+1). fitness_mfea]
    for mode in ["MFEA", "MFEA2", "DE"]:
        MFEA(prob = function_list, POPSIZE = 100*len(function_list) , MAX_GEN=None , MAX_FES = 500000*len(function_list), rmp = 0.3, gen_length = 30, mode = mode).run()

ID: 20
Optimal value:  -98.92802572031052
ID: 21
Optimal value:  -50
Final result: 1: -100.0
Final result: 2: -50.0


KeyboardInterrupt: 

In [ ]:
# from MFEAcode import MFEA 
# MFEA(prob = function_list, POPSIZE = 100*len(function_list) , MAX_GEN=None , MAX_FES = 500000*len(function_list), rmp = 0.3, gen_length = 30, mode = "DE").run()

## 2. DE

In [ ]:
# FEs = 0
# def DE(idx):
#     problem = GNBG(idx)
#     threshold = 1e-8
#     PopulationSize = 100
#     LB = problem.LB 
#     UB = problem.UB 
#     dim = problem.dim
#     X = LB + (UB - LB) * np.random.rand(PopulationSize, dim)
#     fitness = problem.fitness_of_pop(X)
#     Donor = np.empty((PopulationSize, dim))
#     Cr = 0.9
#     F = 0.5
#     FEs = 0
#     BestID = np.argmin(fitness)
#     BestPosition = X[BestID,:]
#     BestValue = fitness[BestID]
#     while True:
#         BestID = np.argmin(fitness)
#         if fitness[BestID] < BestValue:
#             BestPosition = X[BestID, :]
#             BestValue = fitness[BestID]
#         if(abs(BestValue - problem.opt_value) < threshold):
#             print(f'at FE {problem.FE}, found best value : {BestValue}')
#             break
#         # Mutation :
#         R = np.full((PopulationSize, 3), 1)
#         for ii in range(PopulationSize):
#             tmp = np.random.permutation(PopulationSize)
#             tmp = tmp[tmp!=ii]
#             R[ii,:] = tmp[:3]
#         Donor = X[R[:,0],:] + F * ( X[R[:,1],:]  - X[R[:,2],:])
#         # Crossover :
#         OffspringPosition = X.copy()
#         i = np.column_stack(((np.arange(PopulationSize)), np.random.randint(0, dim, size=PopulationSize)))
#         OffspringPosition[i[:,0], i[:,1]] = Donor[i[:,0], i[:,1]]
#         CrossoverBinomial = np.random.rand(PopulationSize, dim) < Cr
#         OffspringPosition[CrossoverBinomial] = Donor[CrossoverBinomial]
#         # Boundary Checking:
#         LB_tmp1 = OffspringPosition < LB 
#         UB_tmp1 = OffspringPosition > UB 
#         LB_tmp2 = ((LB+X)*LB_tmp1)/2
#         UB_tmp2 = ((UB+X)*UB_tmp1)/2
#         OffspringPosition[LB_tmp1] = LB_tmp2[LB_tmp1]
#         OffspringPosition[UB_tmp1] = UB_tmp2[UB_tmp1]
#         OffspringFitness = problem.fitness_of_pop(OffspringPosition)
#         better = OffspringFitness < fitness
#         X[better.flatten()] = OffspringPosition[better.flatten()]
#         fitness[better] = OffspringFitness[better]
#         if problem.FE >= 500000:
#             print(f'at FE {500000}, found best value: {BestValue}')
#             FEs = 0
#             break

In [ ]:
# for i in range(1,25):
#     DE(i)